In [1]:
import pandas as pd
from tensorflow.keras.preprocessing import image

import tensorflow as tf

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet_v2 import ResNet152V2

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import activations
from tqdm import tqdm
import numpy as np

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
df = pd.read_csv('./data/HAM10000_Metadata')

(Warning for Sunlight, increased risk of cancer) Actinic keratoses and intraepithelial carcinoma / Bowen's disease (__akiec__), 0

(Skin cancer) basal cell carcinoma (__bcc__), 1

(HARMLESS) benign keratosis-like lesions (solar lentigines / seborrheic keratoses and lichen-planus like keratoses, __bkl__), 2

(HARMLESS) dermatofibroma (__df__),  3

melanoma (__mel__),  4

(Not entirely harmless) melanocytic nevi (__nv__)   5

(HARMLESS) vascular lesions (angiomas, angiokeratomas, pyogenic granulomas and hemorrhage, __vasc__). 6 


More than 50% of lesions are confirmed through histopathology (histo), the ground truth for the rest of the cases is either follow-up examination (follow_up), expert consensus (consensus), or confirmation by in-vivo confocal microscopy (confocal). The dataset includes lesions with multiple images, which can be tracked by the lesion_id-column within the HAM10000_metadata file.

In [4]:
# Number of cases where we can use LSTM

(df['lesion_id'].value_counts() > 1).sum()

1956

In [5]:
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern


In [6]:
df = df.sample(frac = 1).reset_index()

df.head()

,index,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,8641,HAM_0000851,ISIC_0025074,nv,histo,50.0,female,back,rosendahl
1,7535,HAM_0001246,ISIC_0032822,nv,histo,40.0,female,lower extremity,vidir_modern
2,2843,HAM_0007507,ISIC_0031056,bcc,histo,65.0,female,lower extremity,rosendahl
3,5861,HAM_0000665,ISIC_0030259,nv,follow_up,60.0,male,lower extremity,vidir_molemax
4,9055,HAM_0000836,ISIC_0030311,nv,histo,45.0,female,back,rosendahl


In [7]:
df['localization'].unique()

array(['back', 'lower extremity', 'unknown', 'abdomen', 'trunk',
       'upper extremity', 'foot', 'face', 'chest', 'hand', 'neck',
       'scalp', 'ear', 'genital', 'acral'], dtype=object)

In [8]:
temp = []

for i in range(0, 10015):
    loc = df.iloc[i]['localization']
    if loc == 'abdomen':
        temp.append(tf.one_hot(0, 15))
    elif loc == 'scalp':
        temp.append(tf.one_hot(1, 15))
    elif loc == 'lower extremity':
        temp.append(tf.one_hot(2, 15))
    elif loc == 'trunk':
        temp.append(tf.one_hot(3, 15))
    elif loc == 'upper extremity':
        temp.append(tf.one_hot(4, 15))
    elif loc == 'back':
        temp.append(tf.one_hot(5, 15))
    elif loc == 'neck':
        temp.append(tf.one_hot(6, 15))
    elif loc == 'face':
        temp.append(tf.one_hot(7, 15))
    elif loc == 'chest':
        temp.append(tf.one_hot(8, 15))
    elif loc == 'foot':
        temp.append(tf.one_hot(9, 15))
    elif loc == 'ear':
        temp.append(tf.one_hot(10, 15))
    elif loc == 'unknown':
        temp.append(tf.one_hot(11, 15))
    elif loc == 'hand':
        temp.append(tf.one_hot(12, 15))
    elif loc == 'acral':
        temp.append(tf.one_hot(13, 15))
    elif loc == 'genital':
        temp.append(tf.one_hot(14, 15))

In [9]:
one_hot_loc = np.array(temp)

one_hot_loc

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [10]:
# VGG-16
preprocessed = []

VGG_load = VGG16(weights='imagenet', include_top=False)
model = models.Sequential()
model.add(VGG_load)
model.add(layers.Flatten())


for i in tqdm(range(0, 10015)):
    file = df.iloc[i]['image_id']
    ohl = one_hot_loc[i]
    
    img = image.load_img('./Data/HAM10000_images/' + file + '.jpg', target_size= (224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.vgg16.preprocess_input(img)
    preds = model.predict(img).reshape(25088)

    preprocessed.append(np.concatenate((preds, ohl)))
    
df['VGG16'] = preprocessed

100%|████████████████████████████████████████████████████████████████████████████| 10015/10015 [06:19<00:00, 26.37it/s]


In [11]:
# Inception
preprocessed = []

IV3_load = InceptionV3(weights='imagenet', include_top=False)
model = models.Sequential()
model.add(IV3_load)
model.add(layers.Flatten())

for i in tqdm(range(0, 10015)):
    file = df.iloc[i]['image_id']
    ohl = one_hot_loc[i]
    
    img = image.load_img('./Data/HAM10000_images/' + file + '.jpg', target_size= (299,299))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    preds = model.predict(img).reshape(131072)

    preprocessed.append(np.concatenate((preds, ohl)))
    
df['IV3'] = preprocessed

100%|████████████████████████████████████████████████████████████████████████████| 10015/10015 [08:24<00:00, 19.84it/s]


In [12]:
# ResNet
preprocessed = []

resnet_load = ResNet152V2(weights='imagenet', include_top=False)
model = models.Sequential()
model.add(resnet_load)
model.add(layers.Flatten())


for i in tqdm(range(0, 10015)):
    file = df.iloc[i]['image_id']
    ohl = one_hot_loc[i]    
    
    img = image.load_img('./Data/HAM10000_images/' + file + '.jpg', target_size= (224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.resnet_v2.preprocess_input(img)
    preds = model.predict(img).reshape(100352)

    preprocessed.append(np.concatenate((preds, ohl)))
    
df['resnet'] = preprocessed

100%|████████████████████████████████████████████████████████████████████████████| 10015/10015 [10:14<00:00, 16.29it/s]


In [13]:
labels = []
for i in range(0, 10015):
    dx = df.iloc[i]['dx']
    if dx == 'akiec':
        labels.append(tf.one_hot(0, 7))
    elif dx == 'bcc':
        labels.append(tf.one_hot(1, 7))
    elif dx == 'bkl':
        labels.append(tf.one_hot(2, 7))
    elif dx == 'df':
        labels.append(tf.one_hot(3, 7))
    elif dx == 'mel':
        labels.append(tf.one_hot(4, 7))
    elif dx == 'nv':
        labels.append(tf.one_hot(5, 7))
    elif dx == 'vasc':
        labels.append(tf.one_hot(6, 7))
        
df['one_hot'] = labels

In [14]:
df.head()

,index,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,VGG16,IV3,resnet,one_hot
0,8641,HAM_0000851,ISIC_0025074,nv,histo,50.0,female,back,rosendahl,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.058568966, 0.0, 0.0, 0.18749638, 0.0, 0.377...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T..."
1,7535,HAM_0001246,ISIC_0032822,nv,histo,40.0,female,lower extremity,vidir_modern,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.44673705, 0.0, 0.0, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T..."
2,2843,HAM_0007507,ISIC_0031056,bcc,histo,65.0,female,lower extremity,rosendahl,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.2396596, 0.0, 0.0, 0.0, 0.20554794, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T..."
3,5861,HAM_0000665,ISIC_0030259,nv,follow_up,60.0,male,lower extremity,vidir_molemax,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.01993767, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T..."
4,9055,HAM_0000836,ISIC_0030311,nv,histo,45.0,female,back,rosendahl,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.3947097, 0.7318024, 0.0, 0.29196405, 0.0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T..."


(Warning for Sunlight, increased risk of cancer) Actinic keratoses and intraepithelial carcinoma / Bowen's disease (__akiec__), 0

(Skin cancer) basal cell carcinoma (__bcc__), 1

(HARMLESS) benign keratosis-like lesions (solar lentigines / seborrheic keratoses and lichen-planus like keratoses, __bkl__), 2

(HARMLESS) dermatofibroma (__df__),  3

melanoma (__mel__),  4

(Not entirely harmless) melanocytic nevi (__nv__)   5

(HARMLESS) vascular lesions (angiomas, angiokeratomas, pyogenic granulomas and hemorrhage, __vasc__). 6 


More than 50% of lesions are confirmed through histopathology (histo), the ground truth for the rest of the cases is either follow-up examination (follow_up), expert consensus (consensus), or confirmation by in-vivo confocal microscopy (confocal). The dataset includes lesions with multiple images, which can be tracked by the lesion_id-column within the HAM10000_metadata file.

In [15]:
risk = []

for i in range(0, 10015):
    if df.iloc[i]['dx'] in ['bkl', 'df', 'vasc']:
        risk.append(0.0)
    elif df.iloc[i]['dx'] == 'nv':
        risk.append(1.0)
    elif df.iloc[i]['dx'] == 'akiec':
        risk.append(2.0)
    elif df.iloc[i]['dx'] in ['mel', 'bcc']:
        risk.append(3.0)
        
df['risk'] = risk

df.head()

,index,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,VGG16,IV3,resnet,one_hot,risk
0,8641,HAM_0000851,ISIC_0025074,nv,histo,50.0,female,back,rosendahl,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.058568966, 0.0, 0.0, 0.18749638, 0.0, 0.377...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",1.0
1,7535,HAM_0001246,ISIC_0032822,nv,histo,40.0,female,lower extremity,vidir_modern,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.44673705, 0.0, 0.0, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",1.0
2,2843,HAM_0007507,ISIC_0031056,bcc,histo,65.0,female,lower extremity,rosendahl,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.2396596, 0.0, 0.0, 0.0, 0.20554794, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",3.0
3,5861,HAM_0000665,ISIC_0030259,nv,follow_up,60.0,male,lower extremity,vidir_molemax,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.01993767, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",1.0
4,9055,HAM_0000836,ISIC_0030311,nv,histo,45.0,female,back,rosendahl,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.3947097, 0.7318024, 0.0, 0.29196405, 0.0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",1.0


In [ ]:
import pickle

In [ ]:
pickle_in = open("VGG16.pickle", "rb")
z_vgg = pickle.load(pickle_in)

pickle_in = open("IV3.pickle", "rb")
z_iv3 = pickle.load(pickle_in)

pickle_in = open("resnet.pickle", "rb")
z_res = pickle.load(pickle_in)

In [ ]:
pickle_in = open("one_hot.pickle", "rb")
z_one_hot = pickle.load(pickle_in)

pickle_in = open("risk.pickle", "rb")
z_risk = pickle.load(pickle_in)

In [16]:
VGG_X = np.array(df['VGG16'])
IV3_X = np.array(df['IV3'])
RES_X = np.array(df['resnet'])

In [17]:
#y = np.array(df['one_hot'])

y = np.array(df['risk'])

In [18]:
vgg_model = models.Sequential()
vgg_model.add(layers.Dense(8192, activation='relu'))
vgg_model.add(layers.Dense(4096, activation='relu'))
vgg_model.add(layers.Dense(2048, activation='relu'))
vgg_model.add(layers.Dense(1024, activation='relu'))
vgg_model.add(layers.Dense(512, activation='relu'))
vgg_model.add(layers.Dense(128, activation='relu'))
vgg_model.add(layers.Dense(4))

In [19]:
new_VGG = []

for i in tqdm(range(0, 10015)):
    new_VGG.append(np.array(VGG_X[i]))

new_VGG = np.array(new_VGG)

new_VGG.shape

100%|██████████████████████████████████████████████████████████████████████████| 10015/10015 [00:01<00:00, 6315.51it/s]


(10015, 25103)

In [20]:
new_y = []
for ele in y:
    new_y.append(tf.one_hot(int(ele), 4))#np.array(tf.one_hot(ele, 4)))
    
new_y = np.array(new_y)

new_y.shape

(10015, 4)

In [36]:
new_y.shape

(10015, 4)

In [37]:
vgg_model.compile(optimizer='Adam',
              loss=tf.nn.softmax_cross_entropy_with_logits,
              metrics=['accuracy'])
vgg_model.fit(new_VGG, new_y, epochs=20, validation_split=0.1)

Epoch 1/20
282/282 [==============================] - 6s 20ms/step - loss: 0.1022 - accuracy: 0.9838 - val_loss: 2.1559 - val_accuracy: 0.8114
Epoch 2/20
282/282 [==============================] - 5s 19ms/step - loss: 0.1231 - accuracy: 0.9805 - val_loss: 1.9356 - val_accuracy: 0.8114
Epoch 3/20
282/282 [==============================] - 5s 18ms/step - loss: 0.2403 - accuracy: 0.9570 - val_loss: 1.3855 - val_accuracy: 0.7834
Epoch 4/20
282/282 [==============================] - 5s 18ms/step - loss: 0.2454 - accuracy: 0.9469 - val_loss: 1.2990 - val_accuracy: 0.8004
Epoch 5/20
282/282 [==============================] - 5s 18ms/step - loss: 0.2356 - accuracy: 0.9283 - val_loss: 1.1609 - val_accuracy: 0.8114
Epoch 6/20
282/282 [==============================] - 5s 18ms/step - loss: 0.0837 - accuracy: 0.9828 - val_loss: 1.2351 - val_accuracy: 0.8104
Epoch 7/20
282/282 [==============================] - 5s 18ms/step - loss: 0.0437 - accuracy: 0.9917 - val_loss: 1.4772 - val_accuracy: 0.8114

In [38]:
new_RES = []

for i in tqdm(range(0, 10015)):
    new_RES.append(np.array(RES_X[i]))

new_RES = np.array(new_RES)

new_RES.shape

100%|██████████████████████████████████████████████████████████████████████████| 10015/10015 [00:02<00:00, 3986.37it/s]


(10015, 100367)

In [39]:
res_model = models.Sequential()
#res_model.add(layers.Dense(8192, activation='relu'))
res_model.add(layers.Dense(4096, activation='relu'))
#res_model.add(layers.Dense(2048, activation='relu'))
res_model.add(layers.Dense(1024, activation='relu'))
#res_model.add(layers.Dense(512, activation='relu'))
res_model.add(layers.Dense(128, activation='relu'))
res_model.add(layers.Dense(4))

In [40]:
res_model.compile(optimizer='Adam',
              loss=tf.nn.softmax_cross_entropy_with_logits,
              metrics=['accuracy'])
res_model.fit(new_RES, new_y, epochs=5, validation_split=0.1)

Epoch 1/5
282/282 [==============================] - 9s 32ms/step - loss: 3.3895 - accuracy: 0.6993 - val_loss: 0.6027 - val_accuracy: 0.7505
Epoch 2/5
282/282 [==============================] - 9s 31ms/step - loss: 0.4019 - accuracy: 0.8509 - val_loss: 0.6313 - val_accuracy: 0.7924
Epoch 3/5
282/282 [==============================] - 9s 31ms/step - loss: 0.2073 - accuracy: 0.9267 - val_loss: 0.8644 - val_accuracy: 0.7844
Epoch 4/5
282/282 [==============================] - 9s 31ms/step - loss: 0.1208 - accuracy: 0.9558 - val_loss: 0.8220 - val_accuracy: 0.8174
Epoch 5/5
282/282 [==============================] - 9s 32ms/step - loss: 0.1320 - accuracy: 0.9573 - val_loss: 0.8893 - val_accuracy: 0.7924


In [44]:
new_IV3 = []

for i in tqdm(range(0, 10015)):
    new_IV3.append(np.array(IV3_X[i]))

new_IV3 = np.array(new_IV3)

new_IV3.shape

100%|██████████████████████████████████████████████████████████████████████████| 10015/10015 [00:02<00:00, 3720.49it/s]


MemoryError: Unable to allocate 4.89 GiB for an array with shape (10015, 131087) and data type float32

In [42]:
iv3_model = models.Sequential()
#iv3_model.add(layers.Dense(8192, activation='relu'))
iv3_model.add(layers.Dense(1024, activation='relu'))
#iv3_model.add(layers.Dense(2048, activation='relu'))
iv3_model.add(layers.Dense(256, activation='relu'))
#iv3_model.add(layers.Dense(512, activation='relu'))
#iv3_model.add(layers.Dense(128, activation='relu'))
iv3_model.add(layers.Dense(4))

In [43]:
iv3_model.compile(optimizer='Adam',
              loss=tf.nn.softmax_cross_entropy_with_logits,
              metrics=['accuracy'])
iv3_model.fit(new_IV3, new_y, epochs=5, validation_split=0.1)

ValueError: Data cardinality is ambiguous:
  x sizes: 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 117978, 11

In [ ]:
predict_y_iv3 = []
temp_iv3 = iv3_model.predict(new_IV3)

for i in tqdm(range(0, 10015)):
    pred = np.argmax(temp_iv3[i])
    predict_y_iv3.append(pred)

In [ ]:
predict_y_vgg = []
temp_vgg = vgg_model.predict(new_VGG[-1001:])
print(temp_vgg.shape)
for i in tqdm(range(0, 1001)):
    pred = np.argmax(temp_vgg[i])
    predict_y_vgg.append(pred)

In [ ]:
predict_y_res = []
temp_res = res_model.predict(new_RES)
for i in tqdm(range(0, 10015)):
    pred = np.argmax(temp_res[i])
    predict_y_res.append(pred)

In [ ]:
cm_iv3 = tf.math.confusion_matrix(np.array(df['risk']), np.array(predict_y_iv3))

In [ ]:
cm_vgg = tf.math.confusion_matrix(np.array(df['risk'][-1001:]), np.array(predict_y_vgg))

In [ ]:
cm_res = tf.math.confusion_matrix(np.array(df['risk']), np.array(predict_y_res))

In [ ]:
cm_iv3 = np.array(cm_iv3).astype('float32')
cm_vgg = np.array(cm_vgg).astype('float32')

cm_res = np.array(cm_res).astype('float32')



In [ ]:
cm_iv3[0] = cm_iv3[0] / 1356
cm_iv3[1] = cm_iv3[1] / 6705
cm_iv3[2] = cm_iv3[2] / 326
cm_iv3[3] = cm_iv3[3] / 1627

cm_vgg[0] = cm_vgg[0]/ cm_vgg[0].sum()
cm_vgg[1] = cm_vgg[1]/ cm_vgg[1].sum()
cm_vgg[2] = cm_vgg[2]/ cm_vgg[2].sum()
cm_vgg[3] = cm_vgg[3]/ cm_vgg[3].sum()

cm_res[0] = cm_res[0] / 1356
cm_res[1] = cm_res[1] / 6705
cm_res[2] = cm_res[2] / 326
cm_res[3] = cm_res[3] / 1627

In [ ]:
import seaborn as sns

sns.heatmap(cm_iv3, annot=True)

In [ ]:
sns.heatmap(cm_vgg, annot=True)

In [ ]:
sns.heatmap(cm_res, annot=True)

In [ ]:
feature_vector = []

for i in tqdm(range(0, 10015)):
    sex = 2 if df.iloc[i]['sex'] == 'male' else 1
    age = df.iloc[i]['age']
    loc = df.iloc[i]['localization']
    
    feat = np.array([])#np.array([age])#, sex])
    
    if loc == 'abdomen':
        feat = np.concatenate((feat, tf.one_hot(0, 15)))
    elif loc == 'scalp':
        feat = np.concatenate((feat, tf.one_hot(1, 15)))
    elif loc == 'lower extremity':
        feat = np.concatenate((feat, tf.one_hot(2, 15)))
    elif loc == 'trunk':
        feat = np.concatenate((feat, tf.one_hot(3, 15)))
    elif loc == 'upper extremity':
        feat = np.concatenate((feat, tf.one_hot(4, 15)))
    elif loc == 'back':
        feat = np.concatenate((feat,tf.one_hot(5, 15)))
    elif loc == 'neck':
        feat = np.concatenate((feat,tf.one_hot(6, 15)))
    elif loc == 'face':
        feat = np.concatenate((feat,tf.one_hot(7, 15)))
    elif loc == 'chest':
        feat = np.concatenate((feat,tf.one_hot(8, 15)))
    elif loc == 'foot':
        feat = np.concatenate((feat,tf.one_hot(9, 15)))
    elif loc == 'ear':
        feat = np.concatenate((feat,tf.one_hot(10, 15)))
    elif loc == 'unknown':
        feat = np.concatenate((feat,tf.one_hot(11, 15)))
    elif loc == 'hand':
        feat = np.concatenate((feat,tf.one_hot(12, 15)))
    elif loc == 'acral':
        feat = np.concatenate((feat,tf.one_hot(13, 15)))
    elif loc == 'genital':
        feat = np.concatenate((feat,tf.one_hot(14, 15)))

    feature_vector.append(feat)

In [ ]:
feat_X = np.array(feature_vector)

In [ ]:
feat_X[0].astype('float32')

In [29]:
class_weights = {3:4,  1:1, 2:25, 0:4}

In [30]:
sample_weights = []

for i in range(0, 10015):
    sample_weights.append(class_weights[df.iloc[i]['risk']])
    
df['weight'] = sample_weights

In [31]:
sample_weights = np.array(sample_weights)

In [ ]:
of_model = models.Sequential()

of_model.add(layers.Dense(4))

In [ ]:
of_model.compile(optimizer='Adam',
              loss=tf.nn.softmax_cross_entropy_with_logits,
              metrics=['accuracy'])
history = of_model.fit(feat_X, new_y, epochs=10, validation_split=0.1, sample_weight=sample_weights)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label="train_loss")
plt.plot(history.history['val_loss'], label="val_loss")

plt.legend()

In [ ]:
new_X = []

for i in tqdm(range(0, 10015)):
    feat = np.concatenate((temp_iv3[i] - 9, temp_vgg[i] + 1, temp_res[i] + 6, feature_vector[i]))
    new_X.append(feat)
    
new_X = np.array(new_X)
new_X.shape

In [ ]:
new_X[4]

In [ ]:
new_y.shape

In [ ]:
final_model = models.Sequential()

final_model.add(layers.Dense(64, activation='relu'))
final_model.add(layers.Dense(32, activation='relu'))
final_model.add(layers.Dense(16, activation='relu'))
final_model.add(layers.Dense(8, activation='relu'))
final_model.add(layers.Dense(4))

In [ ]:
final_model.compile(optimizer='Adam',
              loss=tf.nn.softmax_cross_entropy_with_logits,
              metrics=['accuracy'])
final_model.fit(new_X, new_y, epochs=10, validation_split=0.1, shuffle=True)

In [ ]:
predict_y_final = []
temp_final = final_model.predict(new_X)

for i in tqdm(range(0, 10015)):
    pred = np.argmax(temp_final[i])
    predict_y_final.append(pred)

In [ ]:
cm_final = tf.math.confusion_matrix(np.array(df['risk']), np.array(predict_y_final))

cm_final = np.array(cm_final).astype('float32')

cm_final[0] = cm_final[0] / 1356
cm_final[1] = cm_final[1] / 6705
cm_final[2] = cm_final[2] / 326
cm_final[3] = cm_final[3] / 1627

In [ ]:
sns.heatmap(cm_final, annot=True)

# New Original Model

In [ ]:
df = pd.read_csv('./data/HAM10000_Metadata')

In [ ]:
df = df.sample(frac = 1).reset_index()

df.head()

In [ ]:
risk = []

for i in range(0, 10015):
    if df.iloc[i]['dx'] in ['bkl', 'df', 'vasc']:
        risk.append(0.0)
    elif df.iloc[i]['dx'] == 'nv':
        risk.append(1.0)
    elif df.iloc[i]['dx'] == 'akiec':
        risk.append(2.0)
    elif df.iloc[i]['dx'] in ['mel', 'bcc']:
        risk.append(3.0)
        
df['risk'] = risk

df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
fig, axs = plt.subplots(4,1, figsize=(15,30), sharex=True)

sns.histplot(x='localization', data=df[df['risk'] == 0.0], color='green', stat='probability', ax=axs[0]).set_title('risk 0')
sns.histplot(x='localization', data=df[df['risk'] == 1.0], color='yellow', stat='probability', ax=axs[1]).set_title('risk 1')
sns.histplot(x='localization', data=df[df['risk'] == 2.0], color='orange', stat='probability', ax=axs[2]).set_title('risk 2')
sns.histplot(x='localization', data=df[df['risk'] == 3.0], color='red', stat='probability', ax=axs[3]).set_title('risk 3')

In [ ]:
X = []

for i in tqdm(range(0, 10015)):
    file = df.iloc[i]['image_id']
    
    img = image.load_img('./Data/HAM10000_images/' + file + '.jpg', target_size= (150,200))
    img = image.img_to_array(img) / 255.0
    
    
    X.append(img)

In [ ]:
X = np.array(X)

X.shape

In [ ]:
risk = []

for ele in df['risk']:
    risk.append(tf.one_hot(int(ele), 4))
    
y = np.array(risk)
y.shape

In [ ]:
ori_model = models.Sequential()
ori_model.add(layers.Conv2D(64, (3, 3), activation='tanh', input_shape=(150,200,3)))
ori_model.add(layers.BatchNormalization())
ori_model.add(layers.Activation(activations.elu))
ori_model.add(layers.MaxPooling2D(2,2))


ori_model.add(layers.Conv2D(32, (3, 3), activation='tanh'))
ori_model.add(layers.BatchNormalization())
ori_model.add(layers.Activation(activations.elu))
ori_model.add(layers.MaxPooling2D(2,2))


ori_model.add(layers.Conv2D(16, (3, 3), activation='tanh'))
ori_model.add(layers.BatchNormalization())
ori_model.add(layers.Activation(activations.elu))
ori_model.add(layers.MaxPooling2D(2,2))


ori_model.add(layers.Flatten(name="feature_output"))

ori_model.add(layers.Dense(1024, activation='relu'))
ori_model.add(layers.Dense(256, activation='relu'))
ori_model.add(layers.Dense(64, activation='relu'))
ori_model.add(layers.Dense(4))

In [ ]:
ori_model.compile(optimizer='Adam',
              loss=tf.nn.softmax_cross_entropy_with_logits,
              metrics=['accuracy'])
ori_model.fit(X, y, epochs=25, validation_split=0.1, batch_size=16)

In [ ]:
ori_model.save('./models/feature')

In [ ]:
loaded_model = tf.keras.models.load_model("./models/feature")


feature_extractor = tf.keras.Model(
    inputs=loaded_model.inputs,
    outputs=loaded_model.get_layer(name="feature_output").output,
)

In [ ]:
feature_vector = []

for i in tqdm(range(0, 10015)):
    sex = 0 if df.iloc[i]['sex'] == 'male' else 1
    age = df.iloc[i]['age']
    loc = df.iloc[i]['localization']
    
    feat = np.array([sex, age])
    
    if loc == 'abdomen':
        feat = np.concatenate((feat, tf.one_hot(0, 15)))
    elif loc == 'scalp':
        feat = np.concatenate((feat, tf.one_hot(1, 15)))
    elif loc == 'lower extremity':
        feat = np.concatenate((feat, tf.one_hot(2, 15)))
    elif loc == 'trunk':
        feat = np.concatenate((feat, tf.one_hot(3, 15)))
    elif loc == 'upper extremity':
        feat = np.concatenate((feat, tf.one_hot(4, 15)))
    elif loc == 'back':
        feat = np.concatenate((feat,tf.one_hot(5, 15)))
    elif loc == 'neck':
        feat = np.concatenate((feat,tf.one_hot(6, 15)))
    elif loc == 'face':
        feat = np.concatenate((feat,tf.one_hot(7, 15)))
    elif loc == 'chest':
        feat = np.concatenate((feat,tf.one_hot(8, 15)))
    elif loc == 'foot':
        feat = np.concatenate((feat,tf.one_hot(9, 15)))
    elif loc == 'ear':
        feat = np.concatenate((feat,tf.one_hot(10, 15)))
    elif loc == 'unknown':
        feat = np.concatenate((feat,tf.one_hot(11, 15)))
    elif loc == 'hand':
        feat = np.concatenate((feat,tf.one_hot(12, 15)))
    elif loc == 'acral':
        feat = np.concatenate((feat,tf.one_hot(13, 15)))
    elif loc == 'genital':
        feat = np.concatenate((feat,tf.one_hot(14, 15)))

    feature_vector.append(feat)

In [ ]:
feature_vector = np.array(feature_vector)

In [ ]:
feature_vector.shape

In [ ]:
image_features = feature_extractor(X)

In [ ]:
X = []
for img_feat, feat in zip(image_features, feature_vector):
    X.append(np.concatenate((img_feat, feat)))

X = np.array(X)

In [ ]:
model = models.Sequential()

model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(4))

In [ ]:
model.compile(optimizer='Adam',
              loss=tf.nn.softmax_cross_entropy_with_logits,
              metrics=['accuracy'])
model.fit(X, y, epochs=25, validation_split=0.1, batch_size=16, sample_weight=sample_weights)

In [ ]:
df = pd.read_csv('./data/HAM10000_Metadata')

In [ ]:
df = df.sample(frac = 1).reset_index()

df.head()

In [ ]:
risk = []

for i in range(0, 10015):
    if df.iloc[i]['dx'] in ['bkl', 'df', 'vasc']:
        risk.append('no')
    elif df.iloc[i]['dx'] == 'nv':
        risk.append('lo')
    elif df.iloc[i]['dx'] == 'akiec':
        risk.append('md')
    elif df.iloc[i]['dx'] in ['mel', 'bcc']:
        risk.append('hi')
        
df['risk'] = risk

df.head()

In [ ]:
df['image_id'] = df['image_id'].apply(lambda x: x + '.jpg')

df.head()

In [ ]:
# VGG-16
VGG_load = VGG16(weights='imagenet')

model = models.Sequential()
model.add(VGG_load)
model.add(layers.Dense(4))

In [ ]:
data_gen = ImageDataGenerator(
    rotation_range=90, 
    width_shift_range=0.2, 
    height_shift_range=0.2,
    horizontal_flip=True, preprocessing_function = tf.keras.applications.vgg16.preprocess_input,
    vertical_flip=True, validation_split=0.1, zoom_range=0.3)

In [ ]:
df['risk'].value_counts()

In [ ]:
data_gen.flow_from_dataframe(df, directory='./data/HAM10000_images',
                                          x_col='image_id', y_col='risk', target_size=(224,224),
                                          subset='training').class_indices

In [ ]:
data_gen.flow_from_dataframe(df, directory='./data/HAM10000_images',
                                          x_col='image_id', y_col='risk', target_size=(224,224),
                                          subset='validation').class_indices

In [ ]:
class_weights = {'hi':1, 'lo':0.25, 'md':5, 'no':1}

In [ ]:
sample_weights = []

for i in range(0, 10015):
    sample_weights.append(class_weights[df.iloc[i]['risk']])
    
df['weight'] = sample_weights

df.head()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              loss=tf.nn.softmax_cross_entropy_with_logits,
              metrics=['accuracy'])

model.fit(
    x=data_gen.flow_from_dataframe(df, directory='./data/HAM10000_images', weight_col='weight',
                                          x_col='image_id', y_col='risk', target_size=(224,224),
                                          subset='training'),
    validation_data=data_gen.flow_from_dataframe(df, directory='./data/HAM10000_images', weight_col='weight',
                                          x_col='image_id', y_col='risk', target_size=(224,224),
                                          subset='validation'),
    epochs=50)

In [ ]:
feature_vector = []

for i in tqdm(range(0, 10015)):
    sex = 0 if df.iloc[i]['sex'] == 'male' else 1
    age = df.iloc[i]['age']
    loc = df.iloc[i]['localization']
    
    feat = np.array([sex, age])
    
    if loc == 'abdomen':
        feat = np.concatenate((feat, tf.one_hot(0, 15)))
    elif loc == 'scalp':
        feat = np.concatenate((feat, tf.one_hot(1, 15)))
    elif loc == 'lower extremity':
        feat = np.concatenate((feat, tf.one_hot(2, 15)))
    elif loc == 'trunk':
        feat = np.concatenate((feat, tf.one_hot(3, 15)))
    elif loc == 'upper extremity':
        feat = np.concatenate((feat, tf.one_hot(4, 15)))
    elif loc == 'back':
        feat = np.concatenate((feat,tf.one_hot(5, 15)))
    elif loc == 'neck':
        feat = np.concatenate((feat,tf.one_hot(6, 15)))
    elif loc == 'face':
        feat = np.concatenate((feat,tf.one_hot(7, 15)))
    elif loc == 'chest':
        feat = np.concatenate((feat,tf.one_hot(8, 15)))
    elif loc == 'foot':
        feat = np.concatenate((feat,tf.one_hot(9, 15)))
    elif loc == 'ear':
        feat = np.concatenate((feat,tf.one_hot(10, 15)))
    elif loc == 'unknown':
        feat = np.concatenate((feat,tf.one_hot(11, 15)))
    elif loc == 'hand':
        feat = np.concatenate((feat,tf.one_hot(12, 15)))
    elif loc == 'acral':
        feat = np.concatenate((feat,tf.one_hot(13, 15)))
    elif loc == 'genital':
        feat = np.concatenate((feat,tf.one_hot(14, 15)))

    feature_vector.append(feat)
feature_vector = np.array(feature_vector)

feature_vector.shape

In [ ]:
unique, counts = np.unique(feature_vector, return_counts=True, axis=1)

In [ ]:
unique

In [ ]:
feature_extractor = tf.keras.Model(
    inputs=VGG_load.inputs,
    outputs=VGG_load.layers[-4].output,
)

In [ ]:
preprocessed = []

for i in tqdm(range(0, 10015)):
    file = df.iloc[i]['image_id']
    feat_vec = feature_vector[i]    
    
    img = image.load_img('./Data/HAM10000_images/' + file, target_size= (224,224))
    img = image.img_to_array(img)
    # img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.vgg16.preprocess_input(img)
    img_feat = feature_extractor(img)

    preprocessed.append(np.concatenate((img_feat, feat_vec)))
    
preprocessed = np.array(preprocessed)

In [ ]:
ohr = []

for i in range(0, 10015):
    risk = df.iloc[i]['risk']
    
    if risk == 'no':
        ohr.append(tf.one_hot(0, 4))
    elif risk == 'lo':
        ohr.append(tf.one_hot(1, 4))
    elif risk == 'md':
        ohr.append(tf.one_hot(2, 4))
    elif risk == 'hi':
        ohr.append(tf.one_hot(3, 4))
        
ohr = np.array(ohr)

ohr.shape

In [ ]:
ohr

In [ ]:
new_model = models.Sequential()
new_model.add(layers.Dense(4096, activation='relu'))
new_model.add(layers.Dense(2048, activation='relu'))
new_model.add(layers.Dense(512, activation='relu'))
new_model.add(layers.Dense(128, activation='relu'))
new_model.add(layers.Dense(4))

In [ ]:
new_model.compile(optimizer='Adam',
              loss=tf.nn.softmax_cross_entropy_with_logits,
              metrics=['accuracy'])
new_model.fit(preprocessed, ohr, sample_weight=np.array(df['weight']), epochs=10, validation_split=0.1)